# Inferențe în modele probabilistice: Eliminarea variabilelor
    Tudor Berariu, 2017
    Alexandru Sorici, 2024

Laboratorul precedent a prezentat câteva aspecte teoretice despre rețele Bayesiene. În practică, pentru folosirea lor se conturează câteva probleme:

 - estimarea structurii (a numărului de variabile și a legăturilor directe dintre acestea)
 - estimarea parametrilor (a tabelelor de distribuții de probabilități condiționate)
 - inferențe (calculul unor probabilități oarecare având deja structura și parametrii)
 
În laboratorul curent ne vom ocupa *doar* de problema **inferenței** folosind **algoritmul de eliminare a variabilelor**.

## Inferențe în Rețele Bayesiene

Una dintre problemele esențiale legate de modelele probabilistice este calculul unor întrebări generale de tipul:
$$p\left({\bf H}|{\bf E}={\bf e}\right)$$
unde ${\bf H}$ reprezintă o mulțime de variabile neobservate (de regulă, ipoteze/cauze posibile care ar explica observațiile) a căror distribuție de probabilitate este cerută în prezența observațiilor ${\bf E} = {\bf e}$ (_evidence_).

### Calculul probabilităților condiționate

Rescriind ecuația de mai sus astfel:

$$p\left({\bf H}|{\bf E}={\bf e}\right) = \frac{p\left({\bf H}, {\bf E}={\bf e}\right)}{p\left({\bf E}={\bf e}\right)} = \frac{1}{Z} p\left({\bf H}, {\bf E}={\bf e}\right)$$

problema inferenței se reduce la estimarea distribuției comune $P\left({\bf H}, {\bf E} = {\bf e}\right)$ și apoi o valoare oarecare din acestă distribuție va putea fi calculată astfel:

$$p\left({\bf H} = {\bf h} \vert {\bf E} = {\bf e}\right) = \frac{p\left({\bf H} = {\bf h}, {\bf E} = {\bf e}\right)}{\sum_{{\bf h}'}p\left({\bf H} = {\bf h}', {\bf E} = {\bf e}\right)}$$

Folosirea acestei expresii permite următoarea relaxare: găsirea unor valori $\phi\left({\bf H} = {\bf h}, {\bf E} = {\bf e}\right) = \frac{1}{Z} p\left({\bf H} = {\bf h}, {\bf E} = {\bf e}\right) \propto p\left({\bf H} = {\bf h}, {\bf E} = {\bf e}\right)$, ceea ce permite lucrul cu valori nenormalizate.

$$p\left({\bf H} = {\bf h} \vert {\bf E} = {\bf e}\right) = \frac{\phi\left({\bf H} = {\bf h}, {\bf E} = {\bf e}\right)}{\sum_{{\bf h}'}\phi\left({\bf H} = {\bf h}', {\bf E} = {\bf e}\right)}$$

În cele ce urmează vom descrie un algoritm care lucrează cu astfel de valori, pe care le vom numi factori.

**Extra:** Algoritmul se aplică și rețelelor Markov (varianta _neorientată_ a rețelelor Bayesiene, unde se lucrează cu valori nenormalizate peste clici de variabile)

## 1. Factori

Un factor va fi o tabelă de valori peste o colecție de variabile. De exemplu, pentru valorile `A`, `B` și `C` un factor $\phi_{ABC}$ ar putea arăta așa:

    A | B | C | Value
    --+---+---+------
    0 | 0 | 0 | 0.1
    0 | 0 | 1 | 0.9
    0 | 1 | 0 | 0.8
    0 | 1 | 1 | 0.2
    1 | 0 | 0 | 0.7
    1 | 0 | 1 | 0.4
    1 | 1 | 0 | 0.5
    1 | 1 | 1 | 0.5

unde $\phi_{ABC}\left(A=1, B=0, C=0\right) = 0.7$.
   
### 1.1. Reprezentarea factorilor

Un factor va fi reprezentat printr-un tuplu cu nume `(vars, values)` unde `vars` este o listă cu numele variabilelor din factorul respectiv, iar `values` este un dicționar ale cărui chei sunt tupluri de valori ale variabilelor, iar valorile dicționarului reprezintă o valoare numerică.

De exemplu, factorul $\phi_{ABC}$ va fi reprezentat astfel:

```(vars=["A", "B", "C"],
    values={(0, 0, 0): .1, (0, 0, 1): .9, (0, 1, 0): .8, (0, 1, 1): .2,
            (1, 0, 0): .7, (1, 0, 1): .4, (1, 1, 0): .5, (1, 1, 1): .5
    }
)```


In [35]:
from itertools import permutations
from operator import mul
from functools import reduce
import sys
from copy import deepcopy
from collections import namedtuple

Factor = namedtuple("Factor", ["vars", "values"])

def _check_factor(_phi, all_vars, control, control2 = None):
    assert sorted(_phi.vars) == sorted(all_vars), \
        f"Wrong variables: {','.join(_phi.vars):s} instead of {','.join(all_vars):s}"
    assert len(_phi.values) == 2 ** len(all_vars), \
        f"Wrong number of entries in phi.values: {len(_phi.values):d}"
    n = len(all_vars)
    if n > 0:
        for j in range(n + 1):
            vals = [0] * (n - j) + [1] * j
            keys = set([p for p in permutations(vals)])
            p = reduce(mul, [_phi.values[k] for k in keys])
            assert abs(p - control[j]) < 1e-9, \
                "Values for " + str(keys) + " are wrong!"
    else:
        assert abs(_phi.values[()] - control[0]) < 1e-9


def _test_multiply(name1, name2, all_vars, control, verbose=False):
    _phi = eval(f"multiply(deepcopy(phi_{name1:s}), deepcopy(phi_{name2:s}))")
    if verbose:
        print(f"Result of ϕ_{name1:s} * ϕ_{name2:s}:")
        print_factor(_phi)
    sys.stdout.write(f"Testing  ϕ_{name1:s} * ϕ_{name2:s} ... ")
    _check_factor(_phi, all_vars, control)
    print("OK!!")

    
def _test_sum_out(var, name, left_vars, control, verbose=False):
    import sys
    from itertools import permutations
    from operator import mul
    from functools import reduce
    _phi = eval(f"sum_out('{var:s}', phi_{name:s})")
    if verbose:
        print_factor(_phi)
    sys.stdout.write(f"Testing  sum_{var:s} ϕ_{name:s} ... ")
    _check_factor(_phi, left_vars, control)
    print("OK!!")

    
def print_factor(phi, indent="\t"):
    line = " | ".join(phi.vars + ["ϕ(" + ",".join(phi.vars) + ")"])
    sep = "".join(["+" if c == "|" else "-" for c in list(line)])
    print(indent + sep)
    print(indent + line)
    print(indent +sep)
    for values, p in phi.values.items():
        print(indent + " | ".join([str(v) for v in values] + [f"{p:.6f}"]))
    print(indent + sep)

    
# Examples

phi_ABC = Factor(vars=["A", "B", "C"],
                 values={(0, 0, 0): .1, (0, 0, 1): .9, (0, 1, 0): .8, (0, 1, 1): .2,
                         (1, 0, 0): .7, (1, 0, 1): .4, (1, 1, 0): .5, (1, 1, 1): .5})
phi_AB = Factor(vars=["A", "B"], values={(0, 0): .1, (0, 1): .9, (1, 0): .8, (1, 1): .2})
phi_BC = Factor(vars=["B", "C"], values={(0, 0): .2, (0, 1): .8, (1, 0): .5, (1, 1): .5})
phi_A = Factor(vars=["A"], values={(0,): .4, (1,): .6})
phi_C = Factor(vars=["C"], values={(0,): .6, (1,): .8})

print_factor(phi_ABC)
print(f"ϕ(A=1, B=0, C=0) = {phi_ABC.values[(1, 0, 0)]:.3f}")

	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.100000
	0 | 0 | 1 | 0.900000
	0 | 1 | 0 | 0.800000
	0 | 1 | 1 | 0.200000
	1 | 0 | 0 | 0.700000
	1 | 0 | 1 | 0.400000
	1 | 1 | 0 | 0.500000
	1 | 1 | 1 | 0.500000
	--+---+---+---------
ϕ(A=1, B=0, C=0) = 0.700


### 1.2. Multiplicarea a doi factori

Doi factori $\phi_1$ și $\phi_2$ se pot multiplica, obținându-se un nou factor ale cărui valori sunt produse ale valorilor din $\phi_1$ și $\phi_2$. Dacă $\phi_1$ este un factor peste ${\bf X} \cup {\bf Y}$, iar $\phi_2$ este un factor peste ${\bf Y} \cup {\bf Z}$ (${\bf Y}$ reprezintă mulțimea variabilelor comune celor doi factori), atunci:

$$\phi\left(X_1, \ldots\ X_{N_X}, Y_1, \ldots Y_{N_Y}, Z_1, \ldots Z_{N_Z}\right) = \phi_1\left(X_1, \ldots\ X_{N_X}, Y_1, \ldots Y_{N_Y}\right) \cdot \phi_2\left(Y_1, \ldots Y_{N_Y}, Z_1, \ldots Z_{N_Z}\right)$$

De exemplu, fie factorii $\phi_{AB}$ și $\phi_{BC}$:

    --+---+----------          --+---+----------
    A | B | ϕ(A,B)             B | C | ϕ(B,C)
    --+---+----------          --+---+----------
    0 | 0 | 0.100000   <--     0 | 0 | 0.200000
    0 | 1 | 0.900000   !!!     0 | 1 | 0.800000   <--
    1 | 0 | 0.800000           1 | 0 | 0.500000
    1 | 1 | 0.200000           1 | 1 | 0.500000   !!!
    --+---+----------          --+---+----------

Factorul nou se creează combinând toate perechile de intrări din $\phi_{AB}$ și $\phi_{BC}$ pentru care valorile variabilelor comune (în acest caz, $B$) sunt identice.

	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.020000
	0 | 0 | 1 | 0.080000   <--
	0 | 1 | 0 | 0.450000
	0 | 1 | 1 | 0.450000   !!!
	1 | 0 | 0 | 0.160000
	1 | 0 | 1 | 0.640000
	1 | 1 | 0 | 0.100000
	1 | 1 | 1 | 0.100000
	--+---+---+---------


**Funcția `multiply`** : primește doi factori și întoarce rezultatul înmulțirii celor doi.

In [36]:
# Multiplicarea a doi factori:

def multiply(phi1, phi2):
    assert isinstance(phi1, Factor) and isinstance(phi2, Factor)
    
    new_vars, common_idxs = [], {}
    for i, var in enumerate(phi1.vars):
        if var in phi2.vars:
            j = phi2.vars.index(var)
            common_idxs[i]= j # ith var in f1 is the jth var in f2
        else:
            new_vars.append(var)
    new_vars.extend(phi2.vars)  # NV = (V1 \ V2) ++ V2
    new_values = {}
    for vals1, p1 in phi1.values.items():
        for vals2, p2 in phi2.values.items():
            if all([vals1[i] == vals2[j] for (i, j) in common_idxs.items()]):
                vals = [v for (i, v) in enumerate(vals1) if i not in common_idxs] + list(vals2)
                new_values[tuple(vals)] = p1 * p2
    return Factor(vars=new_vars, values=new_values)


In [37]:
print_factor(phi_AB)
print("*")
print_factor(phi_BC)
print("=")
print_factor(multiply(phi_AB, phi_BC))

	--+---+-------
	A | B | ϕ(A,B)
	--+---+-------
	0 | 0 | 0.100000
	0 | 1 | 0.900000
	1 | 0 | 0.800000
	1 | 1 | 0.200000
	--+---+-------
*
	--+---+-------
	B | C | ϕ(B,C)
	--+---+-------
	0 | 0 | 0.200000
	0 | 1 | 0.800000
	1 | 0 | 0.500000
	1 | 1 | 0.500000
	--+---+-------
=
	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.020000
	0 | 0 | 1 | 0.080000
	0 | 1 | 0 | 0.450000
	0 | 1 | 1 | 0.450000
	1 | 0 | 0 | 0.160000
	1 | 0 | 1 | 0.640000
	1 | 1 | 0 | 0.100000
	1 | 1 | 1 | 0.100000
	--+---+---+---------


### 1.3 Eliminarea unei variabile dintr-un factor prin însumare

O variabilă $X_i$ se poate elimina dintr-un factor $\phi$ prin însumarea tuturor valorilor în care celelalte variabile au aceleași valori. Rezultatul este un nou factor $\tau$ al cărui context este dat de toate celelate variabile din $\phi$ în afara lui $X_i$.

Notație: $$\tau \leftarrow \sum_{X_i} \phi$$

$$\tau\left(X_1, \ldots X_{i-1}, X_{i+1}, \ldots, X_N\right) = \sum_{x} \phi \left(X_1, \ldots X_{i-1}, X_i= x, X_{i+1}, \ldots, X_N\right)$$

Prin eliminarea lui $B$ din factorul:
```
--+---+---+----------
A | B | C | ϕ(A,B,C)
--+---+---+----------
0 | 0 | 0 | 0.100000    !!!
0 | 0 | 1 | 0.900000
0 | 1 | 0 | 0.800000    !!!
0 | 1 | 1 | 0.200000
1 | 0 | 0 | 0.700000    <--
1 | 0 | 1 | 0.400000
1 | 1 | 0 | 0.500000    <--
1 | 1 | 1 | 0.500000
--+---+---+----------
```

se obține:

```
--+---+----------
A | C | ϕ(A,C)
--+---+----------
0 | 0 | 0.900000    !!!
0 | 1 | 1.100000
1 | 0 | 1.200000    <---
1 | 1 | 0.900000
--+---+----------
```

**Funcția `sum_out`**: primește o variabilă `var` și un factor `phi` și întoarce un factor nou obținut prin eliminarea prin însumare a variabilei `var`.

In [38]:
def sum_out(var, phi):
    assert isinstance(phi, Factor) and var in phi.vars
    i = phi.vars.index(var)
    new_vars = phi.vars[:i] + phi.vars[(i+1):]
    new_values = {}
    for vals, p in phi.values.items():
        _vals = vals[:i] + vals[(i+1):]
        new_values[_vals] = new_values.get(_vals, 0) + p
    return Factor(vars=new_vars, values=new_values)

In [39]:
# Un exemplu

print("Însumând B afară din")
print_factor(phi_ABC)
print("=")
print_factor(sum_out("B", phi_ABC))

Însumând B afară din
	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.100000
	0 | 0 | 1 | 0.900000
	0 | 1 | 0 | 0.800000
	0 | 1 | 1 | 0.200000
	1 | 0 | 0 | 0.700000
	1 | 0 | 1 | 0.400000
	1 | 1 | 0 | 0.500000
	1 | 1 | 1 | 0.500000
	--+---+---+---------
=
	--+---+-------
	A | C | ϕ(A,C)
	--+---+-------
	0 | 0 | 0.900000
	0 | 1 | 1.100000
	1 | 0 | 1.200000
	1 | 1 | 0.900000
	--+---+-------


### 1.4 Reducerea factorilor conform observațiilor

Factorii inițiali ai unei rețele Bayesiene sunt dați de **tabelele de probabilități condiționate**. Tabelele de valori posibile ale factorului sunt date, așadar, de produsul cartezian al valorilor posibile pentru fiecare variabilă în probabilitatea $p\left(X_i \vert parinti(X_i)\right)$.

Cu toate acestea, variabilele care fac parte din *probe* (_evidence_) vor avea deja o valoare _observată_. Ca atare, vom dori să reducem liniile din tabelele factorilor inițiali la cele care corespund valorilor observate pentru variabilele probă. 

**Funcția `condition_factors`**: Reduce factorii eliminanând intrările ce nu corespund observațiilor făcute.

In [40]:
def condition_factors(Phi : list, Z : dict, verbose=False):
    new_factors = []
    for phi in Phi:
        idxs = {}
        for z, v in Z.items():
            if z in phi.vars:
                idxs[phi.vars.index(z)] = v
        if len(idxs) > 0:
            new_values = {}
            for vals, f in phi.values.items():
                if all([vals[i] == v for (i, v) in idxs.items()]):
                    new_values[vals] = f
            tau = Factor(vars=phi.vars[:], values=new_values)
            if verbose:
                print("\n----------\nNew factor:")
                print_factor(tau)
            new_factors.append(tau)
        else:
            new_factors.append(phi)
    return new_factors

In [41]:
# Un exemplu
print("Aplicand B=0 in factorul")
print_factor(phi_ABC)
print("=>")
print_factor(condition_factors([phi_ABC], {"B": 0})[0])

Aplicand B=0 in factorul
	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.100000
	0 | 0 | 1 | 0.900000
	0 | 1 | 0 | 0.800000
	0 | 1 | 1 | 0.200000
	1 | 0 | 0 | 0.700000
	1 | 0 | 1 | 0.400000
	1 | 1 | 0 | 0.500000
	1 | 1 | 1 | 0.500000
	--+---+---+---------
=>
	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.100000
	0 | 0 | 1 | 0.900000
	1 | 0 | 0 | 0.700000
	1 | 0 | 1 | 0.400000
	--+---+---+---------


## 2. Eliminarea unei variabile dintr-o mulțime de factori

Dându-se o mulțime de factori $\Phi$, dorim să eliminăm variabila $X$. Operația se face prin înlocuirea tuturor factorilor care conțin variabila $X$ cu unul obținut prin (1) factorizare și apoi (2) însumare.

`prod_sum(`$\Phi$ `, ` $X$ `)`
 - $\Phi_{X} \leftarrow \left\lbrace \phi \in \Phi \,:\, X \in \phi \right\rbrace$
 - $\omega \leftarrow \prod_{\phi \in \Phi_{X}} \phi$
 - $\tau \leftarrow \sum_{X} \omega$
 - `return` $\Phi \setminus \Phi_{X} \cup \left\lbrace \tau \right\rbrace$

### Cerința 1 
Implementați funcția `prod_sum` care primește o variabilă `var` și o listă de factori și întoarce noua listă de factori obținută prin eliminarea variabilei `var`. Dacă `verbose` este `True`, atunci afișați factorul nou construit (e util mai târziu pentru a urmări pașii algoritmului).

In [42]:
def prod_sum(var, Phi, verbose=False):
    assert isinstance(var, str) and all([isinstance(phi, Factor) for phi in Phi])
    
    Phi_X = [phi for phi in Phi if var in phi.vars]
    omega = reduce(multiply, Phi_X)
    factor_tau = sum_out(var, omega)

    # Cerinta 1
    
    if verbose:
        print(f"\n----------\nFactor nou dupa eliminarea variabilei {var:s}:")
        print_factor(factor_tau)
    return [phi for phi in Phi if var not in phi.vars] + [factor_tau]

In [43]:
# Un exemplu
print("Elininând B din :")
print_factor(phi_AB)
print("și")
print_factor(phi_BC)
print("=>")
print_factor(prod_sum("B", [phi_AB, phi_BC])[0])

Elininând B din :
	--+---+-------
	A | B | ϕ(A,B)
	--+---+-------
	0 | 0 | 0.100000
	0 | 1 | 0.900000
	1 | 0 | 0.800000
	1 | 1 | 0.200000
	--+---+-------
și
	--+---+-------
	B | C | ϕ(B,C)
	--+---+-------
	0 | 0 | 0.200000
	0 | 1 | 0.800000
	1 | 0 | 0.500000
	1 | 1 | 0.500000
	--+---+-------
=>
	--+---+-------
	A | C | ϕ(A,C)
	--+---+-------
	0 | 0 | 0.470000
	0 | 1 | 0.530000
	1 | 0 | 0.260000
	1 | 1 | 0.740000
	--+---+-------


In [34]:
## Test prod_sum

sys.stdout.write("Testing prod-sum (I) ... ")
result = prod_sum("B", [deepcopy(_phi) for _phi in [phi_A, phi_C, phi_ABC, phi_BC]])
assert len(result) == 3
for _phi in result:
    if sorted(_phi.vars) == ["A", "C"]:
        assert abs(_phi.values[(0, 0)] - 0.42) < 1e-9
        assert abs(_phi.values[(0, 1)] * _phi.values[(1, 0)] - 0.3198) < 1e-9
        assert abs(_phi.values[(1, 1)] - 0.57) < 1e-9
    elif sorted(_phi.vars) == ["A"]:
        assert abs(_phi.values[(0,)] - 0.4) < 1e-9
        assert abs(_phi.values[(1,)] - 0.6) < 1e-9
    elif sorted(_phi.vars) == ["C"]:
        assert abs(_phi.values[(0,)] - 0.6) < 1e-9
        assert abs(_phi.values[(1,)] - 0.8) < 1e-9
print("OK!")

sys.stdout.write("Testing prod-sum (II) ... ")
result = prod_sum("A", [deepcopy(_phi) for _phi in [phi_A, phi_C, phi_ABC, phi_BC]])
assert len(result) == 3
for _phi in result:
    if sorted(_phi.vars) == ["B", "C"]:
        assert abs(_phi.values[(0, 0)] - 0.2) < 1e-9 or abs(_phi.values[(0, 0)] - 0.46) < 1e-9
        assert abs(_phi.values[(0, 1)] * _phi.values[(1, 0)] - 0.4) < 1e-9 or \
               abs(_phi.values[(0, 1)] * _phi.values[(1, 0)] - 0.372) < 1e-9
        assert abs(_phi.values[(1, 1)] - 0.5) < 1e-9 or abs(_phi.values[(1, 1)] - 0.38) < 1e-9
    elif sorted(_phi.vars) == ["C"]:
        assert abs(_phi.values[(0,)] - 0.6) < 1e-9
        assert abs(_phi.values[(1,)] - 0.8) < 1e-9
print("OK!")
print("Prod-Sum seems ok!")

Testing prod-sum (I) ... OK!
Testing prod-sum (II) ... OK!
Prod-Sum seems ok!


## 3. Eliminarea variabilelor

Dându-se o mulțime de factori $\Phi$ și o mulțime de variabile de eliminat $\bf{Z}$, dorim să construim factorul obținut după eliminarea tuturor variabilelor $Z_i$.

`variable_elimination(` $\Phi$ `,` ${\bf Z}$ `)`
 - `for` $Z_i \in {\bf Z}$:
   - $\Phi \leftarrow $ `prod_sum(` $Z_i$ `,` $\Phi$ `)`
 - `return` $\prod_{\phi \in \Phi} \phi$
 
Ordinea în care se iau variabilele din ${\bf Z}$ poate infleunța eficiența algoritmului. (Vezi BONUS.)

### Cerința 2 
Implementați funcția `variable_elimination`. Aceasta trebuie să întoarcă un singur factor. Folosiți argumentul `verbose` și în apelurile funcției `prod_sum`.

In [45]:
def variable_elimination(Phi, Z, verbose=False):
    # Cerinta 2:
    for Z_i in Z:
        Phi = prod_sum(Z_i, Phi, verbose)
    return reduce(multiply, Phi)

In [46]:
## Testing Variable elimination

def _test_variable_elimination(Phi, Vars, left_vars, control, verbose=False):

    
    var_list = '["' + '", "'.join(Vars) + '"]'
    factor_list = '[' + ','.join([f"deepcopy(phi_{name:s})" for name in Phi]) +']'
    name_list = '[' + ','.join([f"ϕ_{name:s}" for name in Phi]) +']'
    _phi = eval(f"variable_elimination({factor_list:s}, {var_list:s})")
    if verbose:
        print_factor(_phi)
    sys.stdout.write(f"Testing  eliminate_var {var_list:s} from {name_list:s} ... ")
    _check_factor(_phi, left_vars, control)
    print("OK!!")

_test_variable_elimination(["A", "C"], ["C"], ["A"], [0.56, 0.84])
_test_variable_elimination(["ABC", "BC", "AB", "A"], ["C", "B"], ["A"], [0.2096, 0.2808])
_test_variable_elimination(["ABC", "BC", "AB", "A"], ["C", "B", "A"], [], [0.4904])
_test_variable_elimination(["ABC", "AB", "BC", "A"], ["A", "B", "C"], [], [0.4904])
_test_variable_elimination(["ABC"], ["A", "B", "C"], [], [4.1])


Testing  eliminate_var ["C"] from [ϕ_A,ϕ_C] ... OK!!
Testing  eliminate_var ["C", "B"] from [ϕ_ABC,ϕ_BC,ϕ_AB,ϕ_A] ... OK!!
Testing  eliminate_var ["C", "B", "A"] from [ϕ_ABC,ϕ_BC,ϕ_AB,ϕ_A] ... OK!!
Testing  eliminate_var ["A", "B", "C"] from [ϕ_ABC,ϕ_AB,ϕ_BC,ϕ_A] ... OK!!
Testing  eliminate_var ["A", "B", "C"] from [ϕ_ABC] ... OK!!


## 4. Realizarea inferențelor în Rețele Bayesiene

$$P\left({\bf H} \vert {\bf E} = {\bf e}\right) = \frac{P\left({\bf H}, {\bf E} = {\bf e}\right)}{P\left(\bf{E = e}\right)}$$

Pentru o rețea Bayesiană definită peste un set de variabile aleatoare ${\bf X}$, unde ${\bf H} \cup {\bf E} \subseteq {\bf X}$, realizarea inferențelor de tipul generic de mai sus se face în următorii pași:
 - Tabelele cu distribuțiile condiționate sunt transformate în factori
 - Factorii ce conțin variabile din ${\bf E}$ sunt reduși la liniile care respectă ${\bf E} = {\bf e}$
 - Fie $\Phi$ mulțimea factorilor astfel obținuți
 - Fie $\phi_{HE}$ factorul obținut prin eliminarea tuturor celorlalte variabile:

     * $\phi_{HE} \leftarrow $ `var_elimination` $\left(\Phi, {\bf X} \setminus \left({\bf H} \cup {\bf E}\right)\right)$

**De notat** că la finalul operației `var_elimination`, factorul $\phi_{HE}$ este **proporțional cu distribuția comună (joint probability)** a variabilelor ${\bf H}$ și ${\bf E=e}$, adică:
$$\phi_{HE} = \frac{1}{Z} p\left({\bf H}, {\bf E=e}\right)$$

Atunci, probabilitatea pe care o căutăm: $$P({\bf H} = {\bf h}| {\bf E}= {\bf e}) = \frac{\phi_{HE}({\bf H}={\bf h})}{\sum_{{\bf H}} \phi_{HE}}$$
 
unde $$\frac{1}{Z} = \frac{1}{\sum_{{\bf H}} \phi_{HE}}$$ 

este **factorul de normalizare** ce permite obținerea valorii de probabilitate pornind de la factorul $\phi_{HE}$.



In [47]:
from random import shuffle

def query(X : list, Y : list, Z : dict, Phi: list, Other=None, verbose=False):
    """
    X - full list of variables
    Y - query variables
    Z - dictionary with observations
    Phi - the list with all factor
    Ohter - an order over variables in X \ (Y U Z); None to pick a random one
    verbose - display factors as they are created
    """

    if verbose:
        print("\n-------------\nInitial factors:")
        for phi in Phi:
            print_factor(phi)

    Phi = condition_factors(Phi, Z, verbose=verbose)  # Condition factors on Z=z

    if Other is None:
        Other = [x for x in X if (x not in Y and x not in Z)]  # Variables that need to be eliminated
        shuffle(Other)
    else:
        assert sorted(Other) == sorted([x for x in X if (x not in Y and x not in Z)])
    if verbose:
        print("\n-------------\nEliminating variables in the following order: " + ",".join(Other))

    phi = variable_elimination(Phi, Other, verbose=verbose)  # Eliminate other variables then Y and Z
    
    # Normalize factor to represent the conditional probability p(Y|Z=z)
    s = sum(phi.values.values())
    prob = Factor(vars=phi.vars, values={k: v / s for (k, v) in phi.values.items()})
    print("\n-----------------\nProbabilitatea ceruta:")
    print_factor(prob)


## 5. Exemplu

Implementăm exemplul din PDF-ul atașat.

In [48]:
phi_a = Factor(vars=["A"], values={(0,): .7, (1,): .3})
phi_b = Factor(vars=["B"], values={(0,): .5, (1,): .5})
phi_c = Factor(vars=["C"], values={(0,): .4, (1,): .6})

phi_d = Factor(vars=["A", "B", "D"],
               values={(0, 0, 0): .75, (0, 0, 1): .25, (0, 1, 0): .7, (0, 1, 1): .3,
                       (1, 0, 0): .6, (1, 0, 1): .4, (1, 1, 0): .2, (1, 1, 1): .8
                      })
phi_e = Factor(vars=["C", "E"],
               values={(0, 0): .25, (0, 1): .75, (1, 0): .75, (1, 1): .25})

phi_f = Factor(vars=["A", "D", "F"],
               values={(0, 0, 0): .6, (0, 0, 1): .4, (0, 1, 0): .4, (0, 1, 1): .6,
                       (1, 0, 0): .7, (1, 0, 1): .3, (1, 1, 0): .8, (1, 1, 1): .2
                      })
phi_g = Factor(vars=["D", "E", "G"],
               values={(0, 0, 0): .1, (0, 0, 1): .9, (0, 1, 0): .2, (0, 1, 1): .8,
                       (1, 0, 0): .5, (1, 0, 1): .5, (1, 1, 0): .4, (1, 1, 1): .6
                      })

all_vars = ["A", "B", "C", "D", "E", "F", "G"]
Phi = [phi_a, phi_b, phi_c, phi_d, phi_e, phi_f, phi_g]

In [49]:
# Algoritmul ar trebui să ajungă la probabilitățile din tabele

# Verificati ca algoritmul "ajunge" corect la valorile din tabele
query(all_vars, ["F"], {"A": 0, "D": 1}, Phi)
query(all_vars, ["G"], {"D": 0, "E": 1}, Phi)


-----------------
Probabilitatea ceruta:
	--+---+---+---------
	F | A | D | ϕ(F,A,D)
	--+---+---+---------
	0 | 0 | 1 | 0.400000
	1 | 0 | 1 | 0.600000
	--+---+---+---------

-----------------
Probabilitatea ceruta:
	--+---+---+---------
	G | D | E | ϕ(G,D,E)
	--+---+---+---------
	0 | 0 | 1 | 0.200000
	1 | 0 | 1 | 0.800000
	--+---+---+---------


In [50]:
# Exemplul din PDF-ul atașat

query(all_vars, ["C", "F"], {"G": 0}, Phi, Other=["E", "B", "A", "D"], verbose=True)



-------------
Initial factors:
	--+-----
	A | ϕ(A)
	--+-----
	0 | 0.700000
	1 | 0.300000
	--+-----
	--+-----
	B | ϕ(B)
	--+-----
	0 | 0.500000
	1 | 0.500000
	--+-----
	--+-----
	C | ϕ(C)
	--+-----
	0 | 0.400000
	1 | 0.600000
	--+-----
	--+---+---+---------
	A | B | D | ϕ(A,B,D)
	--+---+---+---------
	0 | 0 | 0 | 0.750000
	0 | 0 | 1 | 0.250000
	0 | 1 | 0 | 0.700000
	0 | 1 | 1 | 0.300000
	1 | 0 | 0 | 0.600000
	1 | 0 | 1 | 0.400000
	1 | 1 | 0 | 0.200000
	1 | 1 | 1 | 0.800000
	--+---+---+---------
	--+---+-------
	C | E | ϕ(C,E)
	--+---+-------
	0 | 0 | 0.250000
	0 | 1 | 0.750000
	1 | 0 | 0.750000
	1 | 1 | 0.250000
	--+---+-------
	--+---+---+---------
	A | D | F | ϕ(A,D,F)
	--+---+---+---------
	0 | 0 | 0 | 0.600000
	0 | 0 | 1 | 0.400000
	0 | 1 | 0 | 0.400000
	0 | 1 | 1 | 0.600000
	1 | 0 | 0 | 0.700000
	1 | 0 | 1 | 0.300000
	1 | 1 | 0 | 0.800000
	1 | 1 | 1 | 0.200000
	--+---+---+---------
	--+---+---+---------
	D | E | G | ϕ(D,E,G)
	--+---+---+---------
	0 | 0 | 0 | 0.100000
	0 | 0 | 1 |